# 버스 승객 승하차 수 전처리 

### 1. 버스 승객수 월별 시간별 승하차 수 데이터      
추정치를 구하기 위해 하루중 1시간단위로 승객의 승하차 비율을 구함.  

In [1]:
!pip install pymysql
!pip install sqlalchemy

In [2]:
import pandas as pd
import pymysql
from Load import *
from sqlalchemy import *
from module import *
from bus_preprocessing import *

In [3]:
# 버스 승객수 월별 시간별 데이터
bus_station_time_df= pd.read_csv('월별_시간별_버스승객수_승하차_raw.csv', encoding='utf-8-sig')
total_bus_time_df=bus_month_time_passenger(bus_station_time_df)
# csv로 내보내기 
total_bus_time_df.to_csv("월별_시간별_버스승객수_승하차비율_완성본.csv", encoding='UTF-8-sig')

C:\Users\user\AppData\Local\Temp\ipykernel_31684\2383374866.py:2: DtypeWarning: Columns (3,55) have mixed types. Specify dtype option on import or set low_memory=False.
  bus_station_time_df= pd.read_csv('월별_시간별_버스승객수_승하차_raw.csv', encoding='utf-8-sig')
c:\Users\user\Desktop\CM_리팩토링_git용\전처리\bus_preprocessing.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ride_df['승차인원 수']=ride_df.sum(axis=1)
c:\Users\user\Desktop\CM_리팩토링_git용\전처리\bus_preprocessing.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

### 2. 버스 승객 승하차 하루 총 승하차 수 데이터 



수집한 일별_버스승객_승하차 데이터를 데이터프레임으로 불러오고 필요한 컬럼들만 추출한다

In [4]:
daily_bus_passenger_df_1 =pd.read_csv("일별_버스승객_승하차_1_raw.csv", encoding='utf-8-sig')
daily_bus_passenger_df_2 =pd.read_csv("일별_버스승객_승하차_2_raw.csv", encoding='utf-8-sig')
daily_bus_passenger_df_3 =pd.read_csv("일별_버스승객_승하차_3_raw.csv", encoding='utf-8-sig')

In [5]:
daily_bus_passenger_list=[daily_bus_passenger_df_1, daily_bus_passenger_df_2, daily_bus_passenger_df_3]
daily_bus_passenger_df=pd.DataFrame()

for daily_bus_passenger_data in daily_bus_passenger_list:
    daily_bus_passenger_data = daily_bus_passenger(daily_bus_passenger_data)
    daily_bus_passenger_df=pd.concat([daily_bus_passenger_df,daily_bus_passenger_data], axis=0)


In [6]:
daily_bus_passenger_df

,RIDE_PASGR_NUM,ALIGHT_PASGR_NUM,월별_노선_정류장_ID
0,12,0,202101_100_110000327
1,36,5,202101_100_110000335
2,19,20,202101_100_107000013
3,17,35,202101_100_107000014
4,55,80,202101_100_107000015
...,...,...,...
7656029,10,76,202305_8762_113000180
7656030,236,4,202305_8762_113000202
7656031,46,69,202305_8762_113000204
7656032,4,86,202305_8762_113000450


### 3. 월별 시간별 승객수 데이터와 일별 총 승객수 데이터를 병합 후 추정치 구하기    
계산식 : 시간별 승객 승하차 비율 X 일별 총 승객 승하차 수 = 매일 시간별 승객 승하차 수          
병합 기준 : 일별 총 승객 승하차 수 (매일매일 시간별로 승객수를 알아야하므로)

In [7]:
# RIGHT MERGE
daily_time_bus_passenger = total_bus_time_df.merge(daily_bus_passenger_df, on='월별_노선_정류장_ID', how='right')

In [8]:
daily_time_bus_passenger=daily_time_bus_passenger.dropna()
daily_time_bus_passenger['USE_MON']=daily_time_bus_passenger['USE_MON'].astype(int)

In [9]:
# 매일 시간별로 승객수 추정치 구하기 
daily_time_bus_passenger_final=calculate_passenger(daily_time_bus_passenger)   

In [10]:
daily_time_bus_passenger_final.to_csv('일별_시간별_버스승객_승하차_추정치.csv',encoding='UTF-8-sig')